# Coreference Resolution
Coreference resolution is the task of finding all expressions that refer to the same entity in a text. It is an important step for a lot of higher level NLP tasks that involve natural language understanding such as document summarization, question answering, and information extraction. I will be focusing on the following points:
* A simple rule-based system that can achieve results that are very hard to beat.
* Get acquainted with the trickiness of evaluating coref systems, and the current solutions in the field.
* Experiment with two neural approaches for coref to be implemented in PyTorch:
  * A feedforward network that only looks at boolean mention-pair features
  * A fully-neural architecture with embeddings all the way down

In [1]:
!git clone https://github.com/aditya-dl/coreference-resolution.git
%cd coreference-resolution/

Cloning into 'coreference-resolution'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 89 (delta 2), reused 84 (delta 1), pack-reused 0
Unpacking objects: 100% (89/89), done.
/content/coreference-resolution


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 5.1 MB/s 


In [ ]:
import os
import nose
import nltk
nltk.download('averaged_perceptron_tagger')
import torch
import pickle
import sklearn
import xmltodict
import numpy as np
import torch.optim as optim
from nltk.tag import pos_tag
from torch.nn import functional as F
from collections import Counter, defaultdict

%load_ext autoreload
%autoreload 2

In [7]:
from library import coref, coref_rules, coref_features, coref_learning, neural_net, utils

# constants for notebook use
ETA_0 = 0.01

# Part 1: Exploring the data

The core data is in the form of "markables", or "referring expressions", which refer to token sequences that can participate in coreference relations.

Each markable is a namedtuple with five elements:
- string: list of tokens
- entity: the ground truth assignments
- start_token: the index of the first token in the markable with respect to the entire document
- end_token: one plus the index of the last token in the markable
- tags: POS tags corresponding to the tokens in "string"

The ```read_data``` function also returns a list of tokens.

### Loading the dataset
I will explore a dataset of articles from the Wall Street Journal (WSJ) extracted and annotated from the Penn Treebank (PTB).

In [14]:
dev_dir = os.path.join('data','wsj','dev')
train_dir = os.path.join('data','wsj','train')
test_dir = os.path.join('data','wsj','test')

In [15]:
markables, words = coref.read_data('06_wsj_0051.sty', basedir=train_dir)

In [16]:
print('Markable object:', markables[0])
print('Words for markable extracted from text:', words[markables[0].start_token:markables[0].end_token])

Markable object: Markable(string=['Fujitsu', 'Ltd.'], entity='set_3082', start_token=0, end_token=2, tags=['NULL', 'NULL'])
Words for markable extracted from text: ['Fujitsu', 'Ltd.']


In [17]:
# Get all markable "strings" associated with a given entity
sorted(coref.get_markables_for_entity(markables,'set_3082'))

['Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 "Fujitsu , Japan 's No. 1 computer maker",
 'Fujitsu Ltd.',
 'It',
 'The company',
 'The company',
 'We',
 'his company',
 'his company',
 'it',
 'it',
 'it',
 'it',
 'it',
 'it',
 'its',
 'its',
 'the company']

In [18]:
# Showcase a function that takes as input a string, 
# and returns a list of distances to the most recent ground truth antecedent for every time the (case-insensitive)
# input string appears. For example, if the input is "they", it should make a list with one element for each time the word "they"
# appears in the list of markables. Each element should be the distance of the word "they" to the nearest previous mention of the entity that
# "they" references. If the input string is not anaphoric, the distance should be zero. 
coref.get_distances(markables, 'they')

[1, 1, 1, 2, 2]

Now let's compare the typical distances for various mention types.

You can see the most frequent mention types by using the `Counter` class.

In [19]:
Counter([' '.join(markable.string) for markable in markables]).most_common(5)

[('it', 9), ('Fujitsu', 7), ('Japan', 5), ('they', 5), ('NEC', 4)]

In [20]:
coref.get_distances(markables, 'Fujitsu')

[15, 8, 49, 12, 7, 4, 2]

In [21]:
coref.get_distances(markables, 'the company')

[4, 4, 6]

In [22]:
coref.get_distances(markables, 'it') # there are 10 because our counter was case-sensitive

[1, 2, 2, 1, 6, 1, 6, 2, 6, 1]

# 2. Rule-based coreference resolution

I have written a simple coreference classifier, which predicts that each markable is linked to the most recent antecedent which is an exact string match.

The code block below applies this method to the dev set.

In [23]:
exact_matcher = coref_rules.make_resolver(coref_rules.exact_match)

The code above has two pieces:

- ```coref_rules.exact_match()``` is a function that takes two markables, and returns `True` iff they are an exact (case-insensitive) string match
- ```make_resolver()``` is a function that takes a matching function, and returns a function that computes an antecedent list for a list of markables.

Let's run it.

In [24]:
ant_exact = exact_matcher(markables)
print(ant_exact[:50])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 21, 28, 29, 30, 31, 32, 33, 34, 35, 27, 8, 38, 39, 40, 41, 42, 20, 44, 45, 46, 47, 48, 49]


The output is a list of antecedent numbers, $c_i$. 
When $c_i = i$, the markable $i$ has no antecedent: it is the first mention of its entity. In this case, all first 20 mentions are new and don't have antecedents. 

We can test whether these predictions are correct by comparing against the key.

In [25]:
ant_true = coref.get_true_antecedents(markables)
print(ant_true)

[0, 1, 2, 3, 1, 0, 6, 7, 8, 9, 10, 11, 12, 6, 14, 15, 16, 13, 15, 19, 5, 20, 8, 23, 24, 21, 24, 25, 28, 28, 30, 27, 32, 33, 11, 31, 34, 22, 38, 39, 38, 36, 42, 35, 44, 45, 46, 47, 48, 49, 50, 18, 51, 53, 54, 17, 56, 52, 58, 32, 60, 60, 7, 63, 43, 65, 4, 67, 66, 69, 68, 64, 72, 62, 70, 75, 41, 77, 76, 79, 80, 81, 59, 83, 84, 85, 86, 87, 88, 89, 82, 91, 92, 93, 94, 95, 88, 97, 98, 55, 100, 101, 102, 103, 104, 73, 106, 107, 108, 108, 110, 90, 84, 109, 114, 115, 116, 112, 118, 119, 71, 78, 122, 123, 74, 122, 126, 96, 37, 129, 130, 131, 120, 133, 134, 131, 136, 135, 138, 132, 134, 141, 142, 139, 143, 145, 146, 147, 148, 149, 144, 151, 150, 153, 154, 155, 156, 157, 152, 128, 160, 161, 158, 162, 163, 165, 137, 147, 168, 168, 164, 171, 170, 57, 121, 173, 176, 165, 124, 179, 177, 178, 182, 142, 184, 185, 184, 180, 159, 189, 190, 111, 192, 193]


In [26]:
num_correct = sum([c_true == c_predict for c_true, c_predict in zip(ant_true, ant_exact)])
acc = num_correct / len(markables)
print(f'correct: {num_correct}\taccuracy: {acc:.3f}')

correct: 128	accuracy: 0.660


## Evaluation

Coreference can be evaluated in terms of recall, precision, and F-measure. Here is how we will define these terms:

- **True positive**: The system predicts $\hat{c}_i < i$, and $\hat{c}_i$ and $i$ are references to the same entity.
- **False positive**: The system predicts $\hat{c}_i < i$, but $\hat{c}_i$ and $i$ are not references to the same entity.
- **False negative**: There exists some $c_i < i$ such that $c_i$ and $i$ are references to the same entity, but the system predicts either $\hat{c}_i = i$, or some $\hat{c}_i$ which is not really a reference to the same entity that $i$ references.
- Recall = $\frac{tp}{tp + fn}$
- Precision = $\frac{tp}{tp + fp}$
- F-measure = $\frac{2RP}{R+P}$

A couple of things to notice here:

- There is no reward for correctly identifying a markable as non-anaphoric (not having any antecedent), but you do avoid committing a false positive by doing this.
- You cannot compute the evaluation by directly matching the predicted antecedents to the true antecedents. Suppose the truth is $a \leftarrow b, b \leftarrow c$, but the system predicts $a \leftarrow b, a \leftarrow c$: the system should receive two true positives, since $a$ and $c$ are references to the same entity in the ground truth.

In [27]:
f,r,p = coref.evaluate_f(exact_matcher, markables)
print(f'{f:.4f}\t{r:.4f}\t{p:.4f}')

0.5231	0.4096	0.7234


In [28]:
all_markables, all_words = coref.read_dataset(train_dir)

In [29]:
coref.eval_on_dataset(exact_matcher, all_markables)

F: 0.5452	R: 0.4130	P:0.8018


(0.545176110260337, 0.41299303944315546, 0.8018018018018018)

Before optimizing on this simple F-measure (sometimes called F1) and its components, one should be aware that in the real world coreference is evaluated over three other metrics, namely $B^3$, **`CEAF`**, and **`MUC`**. You can read more about them [here](http://www.anthology.aclweb.org/W/W10/W10-4305.pdf).

In [30]:
def coref_metrics(matcher, dataset):
    ants = [matcher(m) for m in dataset]
    b3, ceaf, muc = coref.evaluate_bcm(dataset, ants)
    avg_f1 = np.average([b3, ceaf, muc])
    print(f'B-Cubed: {b3:.4f}\tCEAF: {ceaf:.4f}\tMUC: {muc:.4f}\tAverage: {avg_f1:.4f}')

In [32]:
coref_metrics(exact_matcher, all_markables); #"Average" is the commonly used main metric in state-of-the-art systems.

B-Cubed: 0.5756	CEAF: 0.4551	MUC: 0.5605	Average: 0.5304


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


The reasons for having multiple measures for coref evaluation is manyfold. One of them has to do with the pre-resolution task of *identifying markables*. Since we're only working with pre-extracted markables, that needn't worry us.

The other reason is that different perspectives on coreference matching are equally plausible - we can focus on single correct predictions, or finding the correct clusters for each entity , for example. Each of these is "gameable" by different trivial classifiers.

To witness this problem, I am implementing `coref_rules.singleton_matcher()`, which produces an assignment where each markable has its own entity, and `coref_rules.full_cluster_matcher()`, which assigns all markables to the same entity. Running the metrics against them will demonstrate the problem.

In [33]:
singleton_resolver = coref_rules.make_resolver(coref_rules.singleton_matcher)
coref_metrics(singleton_resolver, all_markables);

B-Cubed: 0.3411	CEAF: 0.0012	MUC: 0.0000	Average: 0.1141


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


MUC has an inherent problem evaluating singleton entities. $B^3$, on the other hand, is extra-generous with them.

In [34]:
full_cluster_resolver = coref_rules.make_resolver(coref_rules.full_cluster_matcher)
coref_metrics(full_cluster_resolver, all_markables);

B-Cubed: 0.0739	CEAF: 0.0304	MUC: 0.5552	Average: 0.2199


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In this case MUC, which is focused on detecting incompatible clusters, is fairly comfortable with the fact that there's only one predicted cluster.
CEAF, a metric which gives low precision very easily, is difficult to score high on.

## Increasing precision

The `exact_match()` function matches everything, including pronouns. This can lead to mistakes:

"Umashanthi ate pizza until she was full. Parvati kept eating until she had a stomach ache."

In this example, both pronouns likely refer to the names that immediately precede them, and not to each other.

The file `coref_rules.py` contains the signature for a function `exact_match_no_pronoun()`, which solves this problem by only predicting matches between markables that are not pronouns.

In [35]:
no_pro_matcher = coref_rules.make_resolver(coref_rules.exact_match_no_pronouns)

In [36]:
f,r,p = coref.eval_on_dataset(no_pro_matcher,all_markables);

F: 0.4551	R: 0.3028	P:0.9158


In [37]:
coref_metrics(no_pro_matcher, all_markables);

B-Cubed: 0.5678	CEAF: 0.4269	MUC: 0.4568	Average: 0.4839


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


Precision has increased, but recall decreased, dragging down the overall F-measure as well as our favorite metrics.

## Increasing recall

Our current matcher is very conservative. Let's try to increase recall. One solution is match on the **head word** of each markable. 

In a CFG parse, the head word is defined by a set of rules: for example, the head of a determiner-noun construction is the noun. In a dependency parse, the head word would be the root of the subtree governing the markable span. But this assumes that the markables correspond to syntactic constituents or dependency subtrees. This is not guaranteed to be true - particularly when there are parsing errors.

Let's start with a much simpler head-finding heuristic: simply select the *last word* in the markable. This handles many cases - but as we will see, not all. The function `match_last_token()` in ```coref_rules.py``` does this. This function matches all cases where the final tokens match.

In [38]:
last_tok_matcher = coref_rules.make_resolver(coref_rules.match_last_token)

In [39]:
coref.eval_on_dataset(last_tok_matcher,all_markables);

F: 0.3994	R: 0.4385	P:0.3666


Recall is up, but precision is back down. To try to increase precision, let's add one more rule: two markables cannot coref if their spans overlap. This can happen with nested mentions, such as "(the president (of the United States))". Under our last-token rule, these two mentions would co-refer, but logically, overlapping markables cannot refer to the same entity. 

The function `match_last_token_no_overlap()` matches any two markables that share the same last token, unless their spans overlap. I am using the `start_token` and `end_token` members of each markable to determine whether they overlap. the final tokens match.

In [40]:
mltno_matcher = coref_rules.make_resolver(coref_rules.match_last_token_no_overlap)
coref.eval_on_dataset(mltno_matcher,all_markables);

F: 0.4911	R: 0.4965	P:0.4858


Both recall and precision increase. Why would recall increase? The restriction does not create any new coreference links, but it changes some incorrect links to correct links. This increases the number of true positives and reduces the number of false negatives.

In [41]:
coref_metrics(mltno_matcher, all_markables);

B-Cubed: 0.5571	CEAF: 0.4610	MUC: 0.5473	Average: 0.5218


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


Almost back to the results from the exact matcher.

## Error analysis

To see whether we can do even better, let's try some error analysis on a specific file.

In [42]:
# predicted antecedent series
markables_17, _ = coref.read_data('17_wsj_0072.sty',basedir=train_dir)
ant = coref_rules.make_resolver(coref_rules.match_last_token_no_overlap)(markables_17)

In [43]:
# let's look at large entities
m2e, e2m = coref.markables_to_entities(markables_17,ant)
big_entities = [ent for ent, vals in e2m.items() if len(vals) > 10]

In [44]:
for entity in big_entities:
    print(f'Entity {entity}: {len(e2m[entity])} mentions')
    print([' '.join(markables_17[idx].string) for idx in e2m[entity]])
    print()

Entity 8: 11 mentions
['Fed', 'Kansas City Fed', 'the Fed', 'the Fed', 'The Fed', 'The report from the Fed', 'the Fed', 'The Philadelphia Fed', 'Fed', 'Fed', 'regional Fed']



## Incorporating parts of speech

One clear mistake is that we are matching ''Kansas City Fed'' to ''The Philadelphia Fed'' and other ''Fed''s. The last token heuristic is the culprit: in this case, the first token is a key disambiguator. Let's try a more syntactically-motivated approach. 

Instead of matching the last token (low precision) or matching on all tokens (low recall), let's try matching on all *content* words. Let's start by including only the following grammatical categories:

- Nouns (proper, common, singular, plural)
- Pronouns (including possessive)
- Adjectives (including comparative and superlative)
- Cardinal numbers

To get these categories, we can call `read_dataset()` again with the optional `tagger` argument, a part of speech tagger. We'll use NLTK for this project, which has a structured perceptron tagger on the [PTB tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). 

In [47]:
all_markables, _ = coref.read_dataset(train_dir, tagger=pos_tag)
all_markables_dev, all_words_dev = coref.read_dataset(dev_dir, tagger=pos_tag)
all_markables_te, all_words_test = coref.read_dataset(test_dir, tagger=pos_tag)

In [48]:
all_markables[1][1]

Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])

As you can see, the markables now have the `tags` member populated with the part-of-speech tags for each token in the `string` field.

We implement a new matcher, `coref_rules.match_on_content()`. This should match $m_a$ and $m_i$ iff all content words are identical. It should also enforce the "no overlap" restriction defined above.

In [49]:
content_matcher = coref_rules.make_resolver(coref_rules.match_on_content)
coref.eval_on_dataset(content_matcher, all_markables)

F: 0.5565	R: 0.4374	P:0.7647


(0.5564575645756457, 0.43735498839907194, 0.7647058823529411)

In [50]:
coref_metrics(content_matcher, all_markables)

B-Cubed: 0.5872	CEAF: 0.4725	MUC: 0.5727	Average: 0.5441


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


Finally getting some headway on those metrics!

In [51]:
# run once, then remove/comment out cell after dir is created
!mkdir predictions

In [52]:
coref.write_predictions(coref_rules.make_resolver(coref_rules.match_on_content),
                        all_markables_dev, 'predictions/rules-dev.preds')

In [53]:
f,r,p = coref.eval_predictions('predictions/rules-dev.preds',all_markables_dev);

F: 0.5463	R: 0.3960	P:0.8806


In [54]:
coref_metrics(content_matcher, all_markables_dev);

B-Cubed: 0.5768	CEAF: 0.4045	MUC: 0.5463	Average: 0.5092


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [55]:
coref.write_predictions(coref_rules.make_resolver(coref_rules.match_on_content), all_markables_te, 
                        'predictions/rules-test.preds')

# Part 3: Machine learning for coreference resolution

We will now implement coreference resolution using the mention-ranking model. Let's start by implementing some features.

We are implementing `coref_features.minimal_features`, using the rules we wrote from `coref_rules.` This should be a function that takes a list of markables, and indices for two mentions, and returns a dict with features and counts. Include the following features:

- `exact-match`
- `last-token-match`
- `content-match`
- `crossover`: value of 1 iff the mentions overlap
- `new-entity`: value of 1 iff i=j

For the first four features, I call the code from coref_rules directly.

In [56]:
min_features = ['exact-match', 'last-token-match', 'content-match', 'crossover', 'new-entity']
for i, markable in enumerate(all_markables[1][:15]):
    print(i, markable)

0 Markable(string=['South', 'Korea'], entity='set_971', start_token=0, end_token=2, tags=['NNP', 'NNP'])
1 Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])
2 Markable(string=['October'], entity='set_973', start_token=11, end_token=12, tags=['NNP'])
3 Markable(string=['the', 'country', "'s", 'economic', 'sluggishness'], entity='set_974', start_token=14, end_token=19, tags=['DT', 'NN', 'POS', 'JJ', 'NN'])
4 Markable(string=['country'], entity='set_971', start_token=15, end_token=16, tags=['NN'])
5 Markable(string=['government', 'figures'], entity='set_975', start_token=22, end_token=24, tags=['NN', 'NNS'])
6 Markable(string=['Wednesday'], entity='set_976', start_token=25, end_token=26, tags=['NNP'])
7 Markable(string=['Preliminary', 'tallies'], entity='set_977', start_token=27, end_token=29, tags=['JJ', 'NNS'])
8 Markable(string=['the', 'Trade', 'and', 'Industry', 'Minist

In [57]:
print(coref_features.minimal_features(all_markables[1],0,1))
print(coref_features.minimal_features(all_markables[1],0,13))
print(coref_features.minimal_features(all_markables[1],13,14))
print(coref_features.minimal_features(all_markables[1],6,6))
print(coref_features.minimal_features(all_markables[1],2,10))

defaultdict(<class 'float'>, {})
defaultdict(<class 'float'>, {'exact-match': 1, 'last-token-match': 1, 'content-match': 1})
defaultdict(<class 'float'>, {'crossover': 1})
defaultdict(<class 'float'>, {'new-entity': 1})
defaultdict(<class 'float'>, {'exact-match': 1, 'last-token-match': 1, 'content-match': 1})


We will now use these features in a simple feedforward neural net. Using pytorch, I implement the `coref_learning.FFCoref` which will be composed of two linear layers separated by a tanh nonlinearity, producing a score for each possible antecedent.

Later we will use this scoring function to select the most probable antecendent.

In [58]:
COREF_FF_HIDDEN = 5 # dimension for hidden layer

In [59]:
torch.manual_seed(1984) # DO NOT CHANGE
coref_ff = coref_learning.FFCoref(min_features, COREF_FF_HIDDEN)

# scores for single mention pairs, no backprop
print(coref_ff(coref_features.minimal_features(all_markables[1],0,1)))
print(coref_ff(coref_features.minimal_features(all_markables[1],0,13)))
print(coref_ff(coref_features.minimal_features(all_markables[1],2,10)))

tensor([-0.1148], grad_fn=<AddBackward0>)
tensor([-0.3763], grad_fn=<AddBackward0>)
tensor([-0.3763], grad_fn=<AddBackward0>)


Given a markable and all its possible antecedents, we now wish to feed all the scores into a softmax layer and attempt the highest possible sum of likelihoods for antecedents representing the correct entity. `FFCoref.score_instance()` does this.

In [60]:
i_scores = coref_ff.score_instance(all_markables[1], coref_features.minimal_features, 13)
print(i_scores)

tensor([[-0.3763, -0.1148, -0.1148, -0.1148, -0.1148, -0.1148, -0.1148, -0.1148,
         -0.1148, -0.1148, -0.1148, -0.1148, -0.1148, -0.0424]],
       grad_fn=<CopySlices>)


In inference time, all we need is to use the above function and report the antecedent for each markable. In training time, we will use an objective based on the **hinge margin-loss** function. Our variant will require the highest-scoring false candidate (according to the true entity annotations) to score lower than the highest-scoring true candidate *by a margin*:

$$L_{m_i} = \{ \text{max}_{a:m_a\notin A(m_i)} s(m_i,m_a) + M - \text{max}_{a:m_a\in A(m_i)} s(m_i,m_a) \}_{+}$$

Where $s$ is the score from the previous deliverable, $A(m)$ denotes the set of true antecedents for $m$, $M$ is our margin, and the $+$ subscript indicates that negative values are replaced by $0$ (since this is a hinge loss).

We implement the helper function `FFCoref.instance_top_scores()` which supplies the arguments for this loss function.
**Note** the special cases where:
- Only true candidates exist (we're in the first cluster) - the trainer will have to skip these. Return `None`s.
- Only false candidates exist - this actually means we're in a new cluster.

In [ ]:
best_true_score, best_false_score = coref_ff.instance_top_scores(all_markables[1], coref_features.minimal_features, 13, 4)
print(torch.cat([best_true_score, best_false_score], 0))

Looks like we're ready to train our classifier!

In [62]:
torch.manual_seed(1984)
coref_ff = coref_learning.FFCoref(min_features, COREF_FF_HIDDEN)
optimizer = optim.SGD(coref_ff.parameters(), lr=ETA_0)
coref_learning.train(coref_ff, optimizer, all_markables, coref_features.minimal_features, margin=1.0, epochs=2)

Loss = 0.862643513347134
Loss = 0.6986558071670488


In [63]:
# training set results
coref_learning.evaluate(coref_ff, all_markables, coref_features.minimal_features)

F: 0.5310	R: 0.4176	P:0.7287


In [64]:
# dev set
coref_learning.evaluate(coref_ff, all_markables_dev, coref_features.minimal_features)

F: 0.5505	R: 0.4027	P:0.8696


In [65]:
# standard metrics on training set
ff_matcher = coref_learning.make_resolver(coref_features.minimal_features, coref_ff)
coref_metrics(ff_matcher, all_markables)

B-Cubed: 0.5869	CEAF: 0.4718	MUC: 0.5723	Average: 0.5436


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [66]:
#test set, students can't run
coref_metrics(ff_matcher, all_markables_te)

B-Cubed: 0.0228	CEAF: 0.0071	MUC: 0.3797	Average: 0.1365


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


We can add more features to try and better capture relations between markables.

I implement distance features in `coref_features.distance_features()`, measuring the mention distance and the token distance. Specifically:

- **Mention distance** is number of intervening mentions between i and j, $i-j$.
- **Token distance** is number of tokens between the start of i and the end of j.

These should be binary features, up to a maximum distance of 10 for tokens / 5 for mentions, with the final feature indicating distance of 10/5 and above, respectively. The desired behavior is shown below.

In [67]:
for i, markable_i in enumerate(all_markables[1][:4]):
    print(i, markable_i)

0 Markable(string=['South', 'Korea'], entity='set_971', start_token=0, end_token=2, tags=['NNP', 'NNP'])
1 Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])
2 Markable(string=['October'], entity='set_973', start_token=11, end_token=12, tags=['NNP'])
3 Markable(string=['the', 'country', "'s", 'economic', 'sluggishness'], entity='set_974', start_token=14, end_token=19, tags=['DT', 'NN', 'POS', 'JJ', 'NN'])


In [68]:
print(coref_features.distance_features(all_markables[1],0,0))
print(coref_features.distance_features(all_markables[1],0,1))
print(coref_features.distance_features(all_markables[1],0,2))
print(coref_features.distance_features(all_markables[1],1,3))
print(coref_features.distance_features(all_markables[1],0,30))

defaultdict(<class 'float'>, {})
defaultdict(<class 'float'>, {'mention-distance-1': 1, 'token-distance-1': 1})
defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-9': 1})
defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-4': 1})
defaultdict(<class 'float'>, {'mention-distance-5': 1, 'token-distance-10': 1})


We implement `coref_features.make_feature_union()`, which should take a list of feature functions, and return a function that computes the union of all features in the list. You can assume the feature functions don't use the same name for any feature.

In [69]:
joint_feats = coref_features.make_feature_union([coref_features.minimal_features,
                                                 coref_features.distance_features])

In [70]:
print(joint_feats(all_markables[1],1,3))
print(joint_feats(all_markables[1],0,3))
print(joint_feats(all_markables[1],0,7))
print(joint_feats(all_markables[1],10,10))

defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-4': 1})
defaultdict(<class 'float'>, {'mention-distance-3': 1, 'token-distance-10': 1})
defaultdict(<class 'float'>, {'mention-distance-5': 1, 'token-distance-10': 1})
defaultdict(<class 'float'>, {'new-entity': 1})


In [71]:
min_features_and_distances = min_features\
                                   + [f'mention-distance-{i}' for i in range(1,6)]\
                                   + [f'token-distance-{i}' for i in range(1,11)]

In [72]:
torch.manual_seed(1984)
coref_ff_w_distances = coref_learning.FFCoref(min_features_and_distances, 50) # we need more hidden units now
optimizer = optim.SGD(coref_ff_w_distances.parameters(), lr=ETA_0)
coref_learning.train(coref_ff_w_distances, optimizer, all_markables, joint_feats, epochs=2)

Loss = 0.7608438846118514
Loss = 0.7099070981127558


In [73]:
coref_learning.evaluate(coref_ff_w_distances, all_markables, joint_feats)

F: 0.4932	R: 0.5023	P:0.4843


In [74]:
ff_w_dist_matcher = coref_learning.make_resolver(joint_feats, coref_ff_w_distances)
coref_metrics(ff_w_dist_matcher, all_markables);

B-Cubed: 0.5579	CEAF: 0.4671	MUC: 0.5421	Average: 0.5224


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


Note that our basic F metric got slightly better, while the average standard metric got slightly worse due to reduced MUC and CEAF.

In [75]:
coref.write_predictions(ff_w_dist_matcher,
                        all_markables_dev,
                        'predictions/ff-dev.preds')
coref.eval_predictions('predictions/ff-dev.preds', all_markables_dev)

F: 0.5088	R: 0.4832	P:0.5373


(0.5088339222614842, 0.48322147651006714, 0.5373134328358209)

In [76]:
coref_metrics(ff_w_dist_matcher, all_markables_dev)

B-Cubed: 0.5702	CEAF: 0.4859	MUC: 0.5442	Average: 0.5334


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [77]:
coref.write_predictions(ff_w_dist_matcher,
                        all_markables_te,
                        'predictions/ff-test.preds')

# Part 4: Sequential Text Represenation

In this section, we will find out whether neural representations of our text can help find coreferents.

The main idea is to run a bidirectional LSTM model, and use the resulting hidden states to form representations of the markables. These will be fed into a feedforward classifier similar to the one from the previous section, except that the match features will also be embedded.

In [78]:
# Preparing the vocabulary for a word-to-index dictionary necessary for the initial embeddings table
vocab = set()
for doc in all_words + all_words_dev + all_words_test:
    vocab.update(doc)
vocab = sorted(list(vocab))
word_to_ix = {w:i for i,w in enumerate(vocab)}
print(len(vocab))

3530


We implement `neural_net.BiLSTMWordEmbedding` as a word embedding lookup table followed by a bi-directional LSTM which runs on a text (here, the entire document) and outputs the hidden state from the LSTM as a contextual embedding for each word in it.

In [79]:
WORD_EMB_DIM = 64
WORD_LSTM_EMB_DIM = 128

In [80]:
torch.manual_seed(1984)
word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.5)
embs = word_lstm(all_words[0])
print(' '.join(all_words[0][:17] + ['...']), '\n')
print(all_words[0][4], '\n', embs[4][0][:5])
print(all_words[0][13], '\n', embs[13][0][:5])

McDermott International Inc. said its Babcock & Wilcox unit completed the sale of its Bailey Controls Operations ... 

its 
 tensor([-0.0129, -0.1065,  0.1406, -0.4480, -0.0298], grad_fn=<SliceBackward0>)
its 
 tensor([-3.4436e-04, -6.9696e-02,  8.9008e-02, -4.5312e-01, -2.8312e-02],
       grad_fn=<SliceBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


We see how the same word type (*its*) is assigned different embeddings based on its context in the document.

## Attention Model

Our markable embeddings will be trained using an **Attention Model** which accepts the embeddings for the words $\{w_i\}$ in a markable and outputs a single vector that "attends" to the single vectors according to what it believes is their importance.
This concept, [originally used](https://arxiv.org/abs/1409.0473) for sequence-to-sequence models such as Machine Translation, is applied for our task as yet another attempt to find the crucial part of a markable, like we did for the head-finding heuristic and for the content-word matching. While those were "hard" techniques, yes-or-no for each token, here we're applying a "soft" weighting that still assigns all the words in the text some significance.

Practically, our model will train a vector parameter of the same embedding size as the BiLSTM output, $\vec{u}$. Each word's embedding in the input span $\vec{e_i}$ will be multiplied (dot-product) with $\vec{u}$ to assign it a scalar weight, $a_i$. Finally each embedding will be multiplied by its normalized (softmaxed) weight $\alpha_i$, and the sum of these weighted vectors will be our markable's output embedding, $\vec{e_m}$:

In [81]:
torch.manual_seed(1984)
attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
mark_embs = [attn_layer(embs, m) for m in all_markables[0]]
for j in [0, 1, 4]:
    print(all_markables[0][j].string, all_markables[0][j].entity, '\n', mark_embs[j][:3])

['McDermott', 'International', 'Inc.'] set_356 
 tensor([0.1076, 0.1651, 0.1864], grad_fn=<SliceBackward0>)
['its'] set_356 
 tensor([-0.0129, -0.1065,  0.1406], grad_fn=<SliceBackward0>)
['its'] set_357 
 tensor([-0.0003, -0.0697,  0.0890], grad_fn=<SliceBackward0>)


We will score a markable pair based on the following features:

1. Each markable's attended embedding
1. A low-dimension embedding for each of the pairwise features we've extracted in the previous sections. Since they are boolean, each will have an embedding for its ''false'' state and one for its ''true'' state.

First, let's implement a quick extractor for positive-valued features from a mention pair.

In [82]:
def get_positive_feats(doc, i, a, feats=coref_features.minimal_features):
    return [k for k,v in feats(doc,i,a).items() if v > 0.0]

In [83]:
get_positive_feats(all_markables[1], 0, 13)

['exact-match', 'last-token-match', 'content-match']

Now we will concatenate all of these embeddings together and use them as input in a two-layer feedforward network (with ReLU nonlinearity) which will produce a scalar score for markable match.

In [84]:
BOOLEAN_FEATURE_DIM = 6
SCORER_HIDDEN_DIM = 164

# starting with just one document
torch.manual_seed(1984)
word_lstm1 = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.5)
embs1 = word_lstm1(all_words[1])
attn_layer1 = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
mkbls1 = all_markables[1]
mark_embs1 = [attn_layer(embs1, m) for m in mkbls1]
scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
scorer(mark_embs1[13], mark_embs1[0], get_positive_feats(mkbls1, 13, 0))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([-0.1786], grad_fn=<AddBackward0>)

We implement `score_instance()` and `instance_top_scores()` in `neural_net.SequentialScorer`. Their purpose is the same as the one in `FFCoref`, but they require the extra embeddings parameter. The former will require some changes to adapt to the different `forward()`, but the latter can be identical to its correlate in `FFCoref` if implemented correctly.

In [85]:
scorer.score_instance(mark_embs1, all_markables[1], 13, coref_features.minimal_features)

tensor([[-0.1786, -0.1996, -0.2356, -0.2084, -0.1870, -0.1862, -0.1949, -0.2230,
         -0.2247, -0.2132, -0.2400, -0.1906, -0.2505, -0.2367]],
       grad_fn=<CopySlices>)

Due to the length of our documents and number of parameters, torch may not work properly on the entire text. We'll truncate the files before we train and use only the minimal pairwise feature space, causing our performance to be suboptimal.

In [86]:
torch.manual_seed(1984)
tr_word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.2)
tr_attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
tr_scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
optimizer = optim.SGD(list(tr_word_lstm.parameters()) + list(tr_attn_layer.parameters()) + list(tr_scorer.parameters()), lr=ETA_0)
neural_net.train(tr_word_lstm, tr_attn_layer, tr_scorer,\
                 optimizer, all_words, all_markables, coref_features.minimal_features, word_limit=150, epochs=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1 complete.
Document losses = 0.22043, 0.21534, 0.27164, 0.17878, 0.37294, 0.43248, 0.34747, 0.24185, 0.07921, 0.27637, 0.23786, 0.19303, 0.33230, 0.23462, 0.10123, 0.25696, 0.17396, 0.32398, 0.15706, 0.05580, 0.29033, 0.48275
Overall loss = 0.24898
Epoch 2 complete.
Document losses = 0.30533, 0.04087, 0.11175, 0.19694, 0.37606, 0.43766, 0.36037, 0.23909, 0.10032, 0.22340, 0.24932, 0.19412, 0.28913, 0.23316, 0.08815, 0.26690, 0.18012, 0.29170, 0.17112, 0.05579, 0.28265, 0.46549
Overall loss = 0.23227
Epoch 3 complete.
Document losses = 0.28406, 0.04547, 0.10289, 0.17440, 0.35293, 0.42351, 0.33969, 0.24050, 0.08158, 0.24350, 0.23826, 0.19589, 0.30743, 0.23147, 0.09214, 0.26338, 0.16493, 0.32297, 0.14797, 0.05483, 0.28206, 0.50596
Overall loss = 0.22909


Let's evaluate on the entire dataset.

In [87]:
tr_resolver = neural_net.evaluate(tr_word_lstm, tr_attn_layer, tr_scorer, all_words, all_markables, coref_features.minimal_features)

F: 0.5017	R: 0.5151	P:0.4890


In [88]:
coref_metrics(tr_resolver, all_markables);

B-Cubed: 0.5514	CEAF: 0.4597	MUC: 0.5390	Average: 0.5167


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [89]:
dv_resolver = neural_net.evaluate(tr_word_lstm, tr_attn_layer, tr_scorer, all_words_dev, all_markables_dev, coref_features.minimal_features)

F: 0.5263	R: 0.5034	P:0.5515


In [90]:
coref_metrics(dv_resolver, all_markables_dev);

B-Cubed: 0.5694	CEAF: 0.4861	MUC: 0.5544	Average: 0.5366


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


## Pretrained Word Embeddings

**Deliverable 4.5**
Implement `utils.initialize_with_pretrained()`. Start by copying from your implementation in PS3.
(0.5 points)

* **Test:** `tests\test_neural_coref.test_pretrain_embeddings_d4_5()`

**Note** that there is a new pretrained file in the `data` folder. Although from the same original source, it is trimmed to the vocabulary in our new dataset, so don't use the same file from PS3. In addition to this attribute, it includes a special token called **&lt;UNK&gt;**. You should use its assigned vector to initialize vectors for all unknown words in the dataset.

Later, if you're interested in improving your model's performance, you may want to know that there are more special tokens with trained vectors in the data file:
* **&lt;S&gt;** signifies the beginning of a sentence.
* **&lt;/S&gt;** signifies the end of a sentence.
* **&lt;PAD&gt;** is used to pad short sentences (this one probably won't be useful).

In [91]:
pret_embs = pickle.load(open('data/pretrained-embeds-coref.pkl', 'rb'))

In [92]:
print(pret_embs['Fujitsu'][:5])

[ 0.34177107 -0.1528549  -0.06971747  0.12536518  0.31670848]


In [93]:
print(pret_embs['<UNK>'][:5])

[-0.2400684   0.0170402  -0.5328812   0.16161029 -0.03450763]


In [94]:
torch.manual_seed(1984)
tr_pt_word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.2)
utils.initialize_with_pretrained(pret_embs, tr_pt_word_lstm)
tr_pt_attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
tr_pt_scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
optimizer = optim.SGD(list(tr_pt_word_lstm.parameters()) + list(tr_pt_attn_layer.parameters()) + list(tr_pt_scorer.parameters()), lr=ETA_0)
neural_net.train(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer,\
                 optimizer, all_words, all_markables, coref_features.minimal_features, word_limit=150, epochs=5)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1 complete.
Document losses = 0.21651, 0.21096, 0.19878, 0.17347, 0.35505, 0.43552, 0.33492, 0.24659, 0.07781, 0.25437, 0.24158, 0.19746, 0.30019, 0.22686, 0.09899, 0.25521, 0.17340, 0.32177, 0.15531, 0.05598, 0.29123, 0.48306
Overall loss = 0.24091
Epoch 2 complete.
Document losses = 0.30625, 0.04299, 0.11356, 0.19089, 0.37243, 0.43567, 0.35710, 0.23797, 0.09520, 0.22210, 0.24407, 0.19255, 0.29286, 0.23245, 0.08275, 0.27424, 0.17821, 0.29517, 0.16738, 0.05564, 0.28156, 0.47456
Overall loss = 0.23153
Epoch 3 complete.
Document losses = 0.28896, 0.04805, 0.10424, 0.16808, 0.34641, 0.42234, 0.33373, 0.23943, 0.07991, 0.23913, 0.23613, 0.19291, 0.30121, 0.23601, 0.07406, 0.28637, 0.17192, 0.29879, 0.16228, 0.05555, 0.28057, 0.47962
Overall loss = 0.22706
Epoch 4 complete.
Document losses = 0.29017, 0.05014, 0.10818, 0.16478, 0.34093, 0.42192, 0.32818, 0.24134, 0.07730, 0.22908, 0.24234, 0.19248, 0.29585, 0.22651, 0.09612, 0.25564, 0.16554, 0.32542, 0.14895, 0.05547, 0.28078, 0.51012

In [95]:
tr_pt_resolver = neural_net.evaluate(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer, all_words, all_markables, coref_features.minimal_features)

F: 0.3240	R: 0.5638	P:0.2273


In [96]:
coref_metrics(tr_pt_resolver, all_markables);

B-Cubed: 0.1567	CEAF: 0.1515	MUC: 0.5113	Average: 0.2732


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [97]:
tr_pt_resolver_dev = neural_net.evaluate(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer, all_words_dev, all_markables_dev, coref_features.minimal_features)
coref.write_predictions(tr_pt_resolver_dev,
                        all_markables_dev,
                        'predictions/nn-dev.preds');
coref_metrics(tr_pt_resolver_dev, all_markables_dev);

F: 0.3596	R: 0.5503	P:0.2671
B-Cubed: 0.2098	CEAF: 0.1831	MUC: 0.5570	Average: 0.3166


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


## Part 5: Domain Adaptation

Our dataset has a second part, which is a corpus of fairy tales, rather than news stories.

Let's take advantage of this setup to see how well our WSJ-trained model does over the fairy tale data, as opposed to a model trained on the fairy tales themselves.

In [98]:
ft_dv_dir = os.path.join('data','tales','dev')
ft_tr_dir = os.path.join('data','tales','train')
ft_te_dir = os.path.join('data','tales','test')

In [99]:
ft_all_markables, ft_all_words = coref.read_dataset(ft_tr_dir, tagger=pos_tag)
ft_all_markables_dev, ft_all_words_dev = coref.read_dataset(ft_dv_dir, tagger=pos_tag)
ft_all_markables_te, ft_all_words_te = coref.read_dataset(ft_te_dir, tagger=pos_tag)

In [100]:
coref.eval_on_dataset(exact_matcher, ft_all_markables)

F: 0.6511	R: 0.5545	P:0.7884


(0.6510840108401084, 0.5545297172533179, 0.7883511074651354)

The exact matcher is getting much higher numbers on this dataset than on WSJ. Let's train an ML system and see the differences. We'll use `FFCoref` from section 3.

In [101]:
torch.manual_seed(1984)
coref_ff_fairy = coref_learning.FFCoref(min_features_and_distances, 50)
optimizer = optim.SGD(coref_ff_fairy.parameters(), lr=ETA_0)
coref_learning.train(coref_ff_fairy, optimizer, ft_all_markables, joint_feats, epochs=2)

Loss = 0.6305409525087488
Loss = 0.602217978526913


In [102]:
# in-domain trained, tested on fairy tale data
coref_learning.evaluate(coref_ff_fairy, ft_all_markables_dev, joint_feats)

F: 0.5970	R: 0.5530	P:0.6486


In [103]:
ft_ff_matcher = coref_learning.make_resolver(joint_feats, coref_ff_fairy)
coref_metrics(ft_ff_matcher, ft_all_markables_dev)

B-Cubed: 0.4960	CEAF: 0.4550	MUC: 0.6517	Average: 0.5342


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


These are the **in-domain** dev set scores for the fairy tale dataset.

Recall that the in-domain numbers for the WSJ portion were the following:

In [104]:
coref_metrics(ff_w_dist_matcher, all_markables_dev)

B-Cubed: 0.5702	CEAF: 0.4859	MUC: 0.5442	Average: 0.5334


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


Now we can see how the **cross-domain** results look. Can a model trained on news data be reliably used in a fairy-tale setting? How about the converse?

In [105]:
# trained on fairy, tested on WSJ
coref_metrics(ft_ff_matcher, all_markables_dev)

B-Cubed: 0.5440	CEAF: 0.4501	MUC: 0.5240	Average: 0.5060


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


In [106]:
# trained on WSJ, tested on fairy
coref_metrics(ff_w_dist_matcher, ft_all_markables_dev)

B-Cubed: 0.5286	CEAF: 0.4951	MUC: 0.6807	Average: 0.5682


/content/coreference-resolution/library/utils.py:168: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  "scipy.optimize.linear_sum_assignment instead.", FutureWarning)


These are lower than in-domain, but surprisingly enough not too low. In other tasks this problem is more acute.